In [1]:
import pathlib
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import random
import os
import re
from dotenv import load_dotenv
from typing import Optional, Union
import dspy
import pickle
from dspy.datasets import Dataset
import pathlib
from sklearn.model_selection import train_test_split
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch
import time
from fuzzywuzzy import process

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
###########
# API KEY #
###########
path_env = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/.env")
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

llm = dspy.OpenAI(model="gpt-3.5-turbo")# "gpt-4o-2024-05-13")

/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/.env


In [4]:
lm = dspy.HFClientTGI(model="meta-llama/Meta-Llama-3-8B ", port=8080, url="http://127.0.0.1")
dspy.settings.configure(lm=lm)

In [22]:
df_samples = pd.read_excel("data/admin_eval_task/tarea_zaragoza.xlsx")
print(df_samples.columns)
df_samples.columns = ["idx","procurement_id", "doc_name", "text", "objetivo"]

Index(['Unnamed: 0', 'procurement_id', 'doc_name', 'text', 'objetivo'], dtype='object')


In [19]:
df_samples.columns

Index(['idx', 'procurement_id', 'doc_name', 'text', 'objetivo'], dtype='object')

In [6]:
class TenderDataset(Dataset):

    def __init__(
        self,
        data_fpath: Union[pathlib.Path, str],
        dev_size: Optional[float] = 0.2,  
        test_size: Optional[float] = 0.2,
        text_key: str = "text",
        label_key: str = "objetivo",
        seed: Optional[int] = 11235,
        *args,
        **kwargs
    ) -> None:
        super().__init__(*args, **kwargs)

        self.labels = []
        self._train = []
        self._dev = []
        self._test = []

        # Read the training data
        train_data = pd.read_excel(data_fpath)

        train_data, temp_data = train_test_split(train_data, test_size=dev_size + test_size, random_state=seed)
        dev_data, test_data = train_test_split(temp_data, test_size=test_size / (dev_size + test_size), random_state=seed)
       
        self._train = [
            dspy.Example({**row}).with_inputs(text_key) for row in self._convert_to_json(train_data)
        ]
        self._dev = [
            dspy.Example({**row}).with_inputs(text_key) for row in self._convert_to_json(dev_data)
        ]
        self._test = [
            dspy.Example({**row}).with_inputs(text_key) for row in self._convert_to_json(test_data)
        ]

    def _convert_to_json(self, data: pd.DataFrame):
        if data is not None:
            return data.to_dict(orient='records')
        return []

In [7]:
# Create dataset
dataset = TenderDataset(
    data_fpath="tarea_zaragoza.xlsx",
    dev_size=0.25,
)

trainset = dataset._train
devset = dataset._dev
testset = dataset._test

In [8]:
from langdetect import detect

def get_lang(df: pd.DataFrame, col_calculate_on: str) -> pd.DataFrame:
    def det(x: str) -> str:
        try:
            lang = detect(x)
        except Exception as e:
            print(e)
            lang = 'Other'
        return lang

    print(f"-- Detecting language...")
    start_time = time.time()

    df['lang'] = df[col_calculate_on].apply(det)

    print(f'-- -- Language detect finished in {(time.time() - start_time)} seconds')

    return df

# Load final_labels from the pickle file
with open('final_labels.pkl', 'rb') as file:
    final_labels = pickle.load(file)

# Convert final_labels to a DataFrame
df_labels = pd.DataFrame(final_labels, columns=['label'])

# Detect the language of each label
df_labels = get_lang(df_labels, 'label')

# Filter the DataFrame to keep only Spanish labels
df_spanish_labels = df_labels[df_labels['lang'] != 'ca']

# Convert the filtered DataFrame back to a list, if necessary
spanish_labels = df_spanish_labels['label'].tolist()

# Display the filtered Spanish labels
#print(spanish_labels)

# To use as HINT
def normalize_string(s):
    # Convert to lower case
    s = s.lower()
    # Remove punctuation and special characters
    s = re.sub(r'[^\w\s]', '', s)
    # Remove extra spaces
    s = re.sub(r'\s+', ' ', s).strip()
    return s

normalized_array = np.array([normalize_string(item) for item in spanish_labels])
unique_dict = {}
for original, normalized in zip(final_labels, normalized_array):
    if normalized not in unique_dict:
        unique_dict[normalized] = original
clean_final_labels = np.array(list(unique_dict.values()))


# Function to normalize the list
def normalize_labels(labels, threshold=85):
    normalized = []
    for label in labels:
        match = process.extractOne(label, normalized, score_cutoff=threshold)
        if match:
            normalized.append(match[0])
        else:
            normalized.append(label)
    return list(set(normalized))

# Normalize the labels
normalized_labels = normalize_labels(clean_final_labels)
print(normalized_labels)

final_labels = ["EJECUCIÓN DEL CONTRATO", "DESCRIPCIÓN DE LOS CONTRATOS", "OBJETIVOS DEL CONTRATO", "OBLIGACIONES DEL CONTRATO", "OBJETO DE LA CONTRATACIÓN"]

-- Detecting language...
-- -- Language detect finished in 3.2378089427948 seconds
['COMISIÓN DE CONTRATACIÓN:', 'OBJETO DE LA CONTRATACIÓN', 'ESPECIFICACIONES DEL CONTRATO', 'EJECUCIÓN DEL CONTRATO', 'PROCEDIMIENTO DE CONTRATACIÓN', 'ÓRGANO DE CONTRATACIÓN PROPONENTE:', 'PER A LA CONTRACTACIÓ DE', "ÁMBIT D'APLICACIÓ DEL CONTRACTE", 'OBJETIVOS DEL CONTRATO.', 'REPRESENTANTE DEL CONTRATO', 'PRESTACIONES DEL CONTRATO', 'IV. EXECUCIÓ DEL CONTRACTE', 'EXPEDIENTE DE CONTRATACIÓN:', 'OBLIGACIONES DEL CONTRATO', 'ADJUDICACION DEL CONTRATO', 'MODIFICACIÓN DEL CONTRATO.', 'CATEGORÍA DEL CONTRATO', 'DESCRIPCIÓN DE LA CONTRATACIÓN']


In [9]:
class PredictObjecto(dspy.Signature):
    """
    Extract the objective of the contract from a document containing the technical specifications of a Spanish public tender. If the objective is not present in the document, return '/'.

    Requirements:

    The extracted text must exclusively consist of words from the document. No additional words are allowed.
    The language of the document must remain unchanged under all circumstances.
    """

    TENDER = dspy.InputField(desc="The document containing the technical specifications of the Spanish public tender.")
    OBJECTIVE = dspy.OutputField(desc="The tender objective, or 'N_A' if not present.")


class PredictModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(PredictObjecto)
        #self.predict = dspy.ChainOfThoughtWithHint(PredictObjecto)
        
    def _process_output(self, text):
        
        if "N_A" in text:
            return "/"
        else:
            return text
      
    def forward(self, text):
        #hint = f"Valid candidates for 'tender objective' may start with {final_labels}."

        pred = self.predict(TENDER=text[0:5000])#, hint=hint

        return dspy.Prediction(objective=self._process_output(pred.OBJECTIVE))

In [10]:
def combined_score(example, pred, trace=None):
    def matching_score(example, pred, trace=None):
        if example.objetivo == "/":
            if pred["objective"] == "/":
                return 1.0
            else:
                return 0.0
        
        predicted_lst = pred["objective"].split()
        gt_lst = example.objetivo.split()
        
        predicted_set = set(predicted_lst)
        gt_set = set(gt_lst)

        intersection = predicted_set.intersection(gt_set)
        union = predicted_set.union(gt_set)
        
        if len(union) == 0:
            return 0.0
        jaccard_similarity = len(intersection) / len(union)
        
        return jaccard_similarity

    def is_in_text_score(example, pred, trace=None):
        if example.objetivo == "/":
            if pred["objective"] == "/":
                return 1.0
            else:
                return 0.0
            
        text_lst = example.text[0:5000].lower().split()
        predicted_lst = pred["objective"].lower().split()

        words_not_in_text = [word for word in predicted_lst if word not in text_lst]
        num_words_not_in_text = len(words_not_in_text)
        
        total_predicted_words = len(predicted_lst)
        score = max(0.0, 1.0 - (num_words_not_in_text / total_predicted_words))
        
        return score

    match_score = matching_score(example, pred, trace)
    text_score = is_in_text_score(example, pred, trace)    
    combined = (0.5 * match_score) + (0.5 * text_score)
    
    return combined

In [11]:
config = dict(max_bootstrapped_demos=3, max_labeled_demos=3, num_candidate_programs=10, max_rounds=1,)
teleprompter = BootstrapFewShotWithRandomSearch(metric=combined_score, **config)
compiled_classifier = teleprompter.compile(PredictModule(), trainset=trainset, valset=devset)

Going to sample between 1 and 3 traces per predictor.
Will attempt to train 10 candidate sets.


Average Metric: 8.722383986214217 / 18  (48.5): 100%|██████████| 18/18 [00:00<00:00, 360.37it/s]
/home/lbartolome/.local/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:137: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 8.722383986214217 / 18  (48.5%)
Score: 48.46 for set: [0]
New best score: 48.46 for seed -3
Scores so far: [48.46]
Best score: 48.46


Average Metric: 8.722383986214217 / 18  (48.5): 100%|██████████| 18/18 [00:00<00:00, 349.59it/s]


Average Metric: 8.722383986214217 / 18  (48.5%)
Score: 48.46 for set: [3]
Scores so far: [48.46, 48.46]
Best score: 48.46


...

Average Metric: 8.070690391180063 / 18  (44.8%)
Score: 44.84 for set: [3]
Scores so far: [48.46, 48.46, 69.39, 44.81, 11.99, 0.0, 55.59, 10.62, 63.94, 46.42, 47.74, 44.84]
Best score: 69.39
Average of max per entry across top 1 scores: 0.6939359413979631
Average of max per entry across top 2 scores: 0.706717118197745
Average of max per entry across top 3 scores: 0.7164636330492901
Average of max per entry across top 5 scores: 0.7192271482861579
Average of max per entry across top 8 scores: 0.7479705141044938
Average of max per entry across top 9999 scores: 0.7550263667485817


 10%|▉         | 4/41 [00:00<00:00, 350.64it/s]

Bootstrapped 2 full traces after 5 examples in round 0.



Average Metric: 6.656283367650821 / 18  (37.0): 100%|██████████| 18/18 [00:00<00:00, 227.64it/s]


Average Metric: 6.656283367650821 / 18  (37.0%)
Score: 36.98 for set: [3]
Scores so far: [48.46, 48.46, 69.39, 44.81, 11.99, 0.0, 55.59, 10.62, 63.94, 46.42, 47.74, 44.84, 36.98]
Best score: 69.39
Average of max per entry across top 1 scores: 0.6939359413979631
Average of max per entry across top 2 scores: 0.706717118197745
Average of max per entry across top 3 scores: 0.7164636330492901
Average of max per entry across top 5 scores: 0.7192271482861579
Average of max per entry across top 8 scores: 0.7479705141044938
Average of max per entry across top 9999 scores: 0.7550263667485817
13 candidate programs found.


In [12]:
tests = []
for el in testset:
    output = compiled_classifier(el.text)
    tests.append([el.text[0:5000],el.objetivo, output["objective"], combined_score(el,output)])

In [17]:
results = pd.DataFrame(tests, columns=["TEXT","GROUND", "PREDICTED", "METRIC"])
results

TEXT  \
0   1ena aeropuerto de tenerife norte ciudad de la laguna suministro de consumibles diversos aena suministro de consumibles diversos reropuerto de tenerife norte departamento de gabinete de dirección ciudad de la laguna página 2 de 11 aena suministro de consumibles diversos reropuerto de tenerife norte departamento de gabinete de dirección ciudad de la laguna el presente pliego de prescripciones técnicas en adelante ppt tiene por objeto definir las condiciones por las que se ha de regir el expediente encargado de suministrar diferentes consumibles al aeropuerto de tenerife norteciudad de la laguna los consumibles a suministrar son los listados en el anexo de este documento el plazo de ejecución será la menor de estas dos fechas un 1 año cuando se haya agotado el presupuesto anual disponible licitado véase ap3 importe total el importe total de este suministro no podrá exceder de trece mil euros 1300000 impuestos no incluidos el adjudicatario presentará su oferta económica con los siguientes requisitos oferta global al importe de licitación ya que este expediente no admite bajas en el total del importe de licitación ni reducción del plazo de ejecución en documento separado indicará la baja lineal ofertada sobre todos los precios unitarios de los artículos igual para todos los productos contemplados en el anexo relación de materiales objeto del suministro del presente documento la forma de proceder yo gestionar este suministro será acordada por el responsable del expediente de aena sme sa y el coordinador de la empresa adjudicataria en todo caso el responsable del expediente propondrá la forma de prestación del suministro petición de materiales consultas técnicas plazo de entrega lugar de entrega forma y fecha de presentación de albaranes para certificaciones etc por tanto la empresa adjudicataria se compromete a aceptar que la ejecución del suministro pueda ser a cualquier hora del día previéndose a priori que éstos sean entregados en las instalaciones del aeropuerto en cualquier día laborable de la semana se consideran 2 tipos de plazo de entrega e productos incluidos en el anexo y en stock dos 2 días desde la recepción de la solicitud por parte del director del expediente o de la persona en la que delegue página 3 de 11 aena suministro de consumibles diversos reropuerto de tenerife norte departamento de gabinete de dirección ciudad de la laguna productos incluidos en el anexo y que el adjudicatario demuestre que no está stock tres 3 semanas desde la recepción de la solicitud por parte del director del expediente o de la persona en la que delegue la entrega deberá realizarse en las oficinas de aena del aeropuerto de tenerife norteciudad de la laguna al propio director de expediente o a la persona a la que éste designe la firma de este expediente con el adjudicatario no supone la exclusividad para la adquisición de los materiales objeto de este suministro el objeto de este expediente será el de suministro de diversos consumibles la necesidad partirá del director del expediente sin embargo a medida que el tiempo avance el adjudicatario adquirirá más conciencia de las necesidades de las dependencias por lo que su aportación será de gran ayuda se espera por tanto proactividad y colaboración todos los elementos detallados en el anexo corresponderán a primeras marcas existentes en el mercado el adjudicatario deberá realizar un trabajo de análisis de los diferentes precios y productos disponibles en ningún caso puede limitarse a buscar el primer proveedor que se le presente debiendo analizar diferentes opciones contrastando asimismo las calidades ofertadas en ningún caso aena gestionará transportes ni despachará con transportistas debiendo ser esta una responsabilidad del adjudicatario al igual que sucede con las gestiones aduaneras y sus importes el director del expediente o persona en quien delegue recepcionará la mercancía en las instalaciones del aeropuerto junto con el adjudicatario comprobando in situ la idoneidad del su

In [14]:
results["METRIC"].mean()

0.7199627557398761